In [1]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    print('You need to pip install google-cloud-service-usage')
    !pip install google-cloud-service-usage -q
    print('installed google-cloud-service-usage')
    

In [2]:
try:
    import google_cloud_pipeline_components
except ImportError:
    print('You need to pip install google-cloud-pipeline-components')
    !pip install google-cloud-pipeline-components -q
    print('installed google-cloud-pipeline-components')

In [3]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'poc-example-ds'

In [4]:
LOCATION = 'europe-west1'
EXPERIMENT = '01a'
SERIES = '01'


## Import a Datasource to BigQuery

In [5]:
# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud'

# Data source for this series of notebooks: Described above
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'


# packages:
from google.cloud import bigquery
from google.cloud import storage
# clients:
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)
# parameters:
BUCKET = PROJECT_ID

In [6]:
# Initialize a client
# If project_id is None, it will use the default project from your gcloud config
client = storage.Client(project=PROJECT_ID)

# Get a bucket object
bucket = client.bucket(BUCKET)

# Check if the bucket exists
if bucket.exists():
    print(f"Bucket '{BUCKET}' already exists.")
else:
    print(f"Bucket '{BUCKET}' does not exist. Creating it in location '{LOCATION}'...")
    # Create the bucket
    bucket.create(location=LOCATION)
    print(f"Bucket '{BUCKET}' created successfully.")

Bucket 'poc-example-ds' already exists.


In [7]:
# store datarouice in GCS bucket

file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"

bucketDef = gcs.bucket(BUCKET)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Creating the export ...')
    extract.result()
    print(f'Exported the table to: gs://{bucketDef.name}/{file}')
list(bucketDef.list_blobs(prefix = f'{SERIES}/{EXPERIMENT}'))
print(f'Review the files in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')

The file has already been created at: gs://poc-example-ds/01/01a/data/fraud.csv
Review the files in the console here:
https://console.cloud.google.com/storage/browser/poc-example-ds/01;tab=objects&project=poc-example-ds


## Create BigQuery Dataset

In [8]:
# List BigQuery datasets in the project:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)
# Create the dataset if missing:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = LOCATION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)
# List BigQuery datasets in the project:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

example_composer_workshop
example_dataproc_workshop
fraud
looker_bq_optimization
example_composer_workshop
example_dataproc_workshop
fraud
looker_bq_optimization


In [9]:
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

The table already exists: poc-example-ds.fraud.fraud


In [10]:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")

Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=poc-example-ds


In [11]:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,8748,-1.070416,0.304517,2.777064,2.154061,0.254450,-0.448529,-0.398691,0.144672,1.070900,...,-0.122032,-0.182351,0.019576,0.626023,-0.018518,-0.263291,-0.198600,0.098435,0.00,0
1,27074,1.165628,0.423671,0.887635,2.740163,-0.338578,-0.142846,-0.055628,-0.015325,-0.213621,...,-0.081184,-0.025694,-0.076609,0.414687,0.631032,0.077322,0.010182,0.019912,0.00,0
2,28292,1.050879,0.053408,1.364590,2.666158,-0.378636,1.382032,-0.766202,0.486126,0.152611,...,0.083467,0.624424,-0.157228,-0.240411,0.573061,0.244090,0.063834,0.010981,0.00,0
3,28488,1.070316,0.079499,1.471856,2.863786,-0.637887,0.858159,-0.687478,0.344146,0.459561,...,0.048067,0.534713,-0.098645,0.129272,0.543737,0.242724,0.065070,0.023500,0.00,0
4,31392,-3.680953,-4.183581,2.642743,4.263802,4.643286,-0.225053,-3.733637,1.273037,0.015661,...,0.649051,1.054124,0.795528,-0.901314,-0.425524,0.511675,0.125419,0.243671,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142396,87209,-0.024950,1.554469,1.587469,4.108145,1.293879,1.075116,1.097120,-0.314708,-2.246136,...,-0.231031,-0.369423,-0.109470,0.730357,-0.533616,-0.011651,-0.314704,-0.284703,0.78,0
142397,163484,-0.639453,1.753366,-1.503488,-0.370006,0.918963,-0.455540,0.052645,-2.587461,-0.909486,...,-0.935817,1.405463,0.023547,0.626422,-0.389439,-0.209657,0.051996,0.206556,0.78,0
142398,158709,2.124206,-0.013996,-1.597990,0.205808,0.299265,-0.975089,0.242633,-0.362167,0.644597,...,0.199132,0.748822,-0.084798,-0.617488,0.360686,-0.067693,-0.020362,-0.065791,0.78,0
142399,43587,-3.223180,3.329060,-0.089129,0.236120,-0.636745,-0.916207,0.488061,-0.033712,1.676460,...,-0.061234,0.294829,-0.022459,0.334070,0.069616,-0.452110,-0.465946,-0.100211,0.78,0


In [12]:
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [13]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()

,splits,Count,Percentage
0,TEST,28481,10.000105
1,VALIDATE,28333,9.948140
2,TRAIN,227993,80.051754


In [15]:
query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()
data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,10910,1.084899,0.199359,1.606916,2.880893,-0.688179,0.627886,-0.844409,0.320399,1.650859,...,0.107861,-0.032271,0.144011,-0.046772,0.021505,0.022237,0.0,0,c5c99f3f-a2ad-43aa-b83b-2159c417d1c5,TEST
1,81133,-0.928037,0.716264,2.196440,-0.129625,-0.954927,-0.258630,-0.284593,0.260785,0.445548,...,-0.115150,0.462642,-0.384294,1.018291,-0.493084,-0.185673,0.0,0,2d953a2f-936e-4ce2-9f7b-a41b11f2cfcc,TEST
2,147639,-1.717979,1.817976,1.400821,4.351664,-0.000967,1.182287,-0.760404,1.377329,-2.052576,...,-0.084482,0.686646,-0.053110,0.034509,0.280359,-0.026721,0.0,0,3a90c465-ba87-434a-be09-f2400f581ecd,TEST
3,63654,-4.883327,-6.110344,1.313198,1.453478,6.203022,-5.562701,-3.636599,0.401356,0.442138,...,1.191681,0.718352,-0.225150,0.255367,-0.133263,0.303217,0.0,0,a04a2bc1-a6a9-4f8a-b39a-5c7ae74acda7,TEST
4,113822,1.957669,0.382698,-0.559121,3.789147,0.619981,0.889021,-0.128089,0.038456,-0.358739,...,-0.056101,-1.022867,0.270033,0.198782,0.007151,-0.055804,0.0,0,3a481b6c-c5f2-4588-aaf9-1aa51adf6443,TEST
